In [1]:
#import libraries
import requests
import pandas as pd
import time
import numpy as np 

In [8]:
#API call for top players
def get_top_player_ids():
    API_KEY = "RGAPI-4912c48d-414b-4c5f-9412-df8f28839e03"
    url = "https://na1.api.riotgames.com/tft/league/v1/challenger?api_key="+API_KEY
    response_top_players = requests.get(url).json()

    summonerIds = []
    for entry in response_top_players['entries']:
        summonerIds.append(entry['summonerId'])
    print("Completed retriving summonerIds")
    return summonerIds

In [9]:
#API call for the top player's list of match ids
def get_matches_ids():
    API_KEY = "RGAPI-4912c48d-414b-4c5f-9412-df8f28839e03"
    max_calls = 2
    puuids = []
    summonerIds = get_top_player_ids()
    start_time = time.time()
    time.sleep(120)
    
    for id in summonerIds:
        if max_calls % 100 == 0:
            time.sleep(120)
        elif max_calls % 20 == 0:
            time.sleep(2)
        url = "https://na1.api.riotgames.com/tft/summoner/v1/summoners/" + id + "?api_key="+API_KEY
        response_top_players_id = requests.get(url).json()
        #if len(response_top_players_id.keys()) > 2:
        puuids.append(response_top_players_id['puuid'])
        max_calls += 1

    max_calls = 1
    match_ids = []
    time.sleep(120)
    
    for id in puuids:
        if max_calls % 100 == 0:
            time.sleep(120)
        elif max_calls % 20 == 0:
            time.sleep(2)
        url = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + id + "/ids?start=0&count=20&api_key="+API_KEY
        response_top_players_match_id = requests.get(url).json()
        #if isinstance(response_top_players_match_id, list):
        match_ids.extend(response_top_players_match_id)
    match_ids = list(dict.fromkeys(match_ids))
    
    end_time = time.time()
    time_lapsed = end_time - start_time
    print("Completed retriving matchIds " + str(time_lapsed) + " seconds")
    return match_ids

In [10]:
#API call for matches
def get_matches():
    API_KEY = "RGAPI-4912c48d-414b-4c5f-9412-df8f28839e03"
    max_match_calls = 1
    match_infomation = []
    match_ids = get_matches_ids()
    start_time = time.time()
    time.sleep(120)
    
    for id in match_ids:
        if max_match_calls % 100 == 0:
            time.sleep(120)
        elif max_match_calls % 20 == 0:
            time.sleep(2)
        url = "https://americas.api.riotgames.com/tft/match/v1/matches/" + id + "/?api_key="+API_KEY
        response_matches = requests.get(url).json()
        #if len(response_matches.keys()) > 2:
        match_infomation.append(response_matches)
        max_match_calls += 1
    
    end_time = time.time()
    time_lapsed = end_time - start_time
    print("Completed retriving match_infomation after " + str(time_lapsed) + " seconds")
    return match_infomation

In [5]:
#Function that returns the game outcomes 
def get_units(df, match_infomation):
    game = 0
    for match in match_infomation:
        participants = match['info']['participants']
        for participant in participants:
            units = participant['units']
            total_units = 0
            for unit in units:
                if "Nomsy" in unit['character_id']:
                    total_units+=1
                elif ("Dragon" in unit['character_id']) | ("AoShin" in unit['character_id']) | ("AurelionSol" in unit['character_id']) | ("Shyvana" in unit['character_id']): 
                    total_units+=2
                else:
                    total_units+=1
            for unit in units:
                unit_name = unit['character_id']
                num_items = len(unit['items'])
                tier = unit['tier']
                placement = participant['placement']

                df = df.append({'unit_name': unit_name,
                                'num_items': num_items,
                                'tier': tier,
                                'placement': placement,
                               'game': game,
                               'total_units': total_units}, ignore_index=True)
        game += 1
    return df

In [11]:
#Main
df = pd.DataFrame(columns=["unit_name", "num_items", "tier", "placement", "game", "total_units"])
match_infomation = get_matches()
df = get_units(df, match_infomation)
df.to_csv("game_outcomes.csv")

Completed retriving summonerIds
Completed retriving matchIds 291.8899929523468 seconds
Completed retriving match_infomation after 1073.442409992218 seconds


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

# get dummy data 
df = pd.read_csv("game_outcomes.csv")

# train test split 
from sklearn.model_selection import train_test_split
df = pd.read_csv("game_outcomes.csv")
num_games = df.iloc[df.index[-1]]['game']
df_train = df.loc[df['game'] % 5 != 0]
df_test = df.loc[df['game'] % 5 == 0]
y = df_train.placement.values
X = pd.get_dummies(df_train.drop(['placement', 'game'], axis =1))
y_test = df_test.placement.values
y_test_games = df_test.game.values
X_test = pd.get_dummies(df_test.drop(['placement', 'game'], axis =1))

from sklearn.utils import shuffle
X_train, y_train = shuffle(X, y, random_state=0)

# random forest 
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

rf.fit(X_train,y_train)

num_incorrect = 0
games = 0
for i in range(5, num_games, 5):
    current_placement = 0
    indicies = np.where(y_test_games == i)
    avg = 0
    if (y[indicies] == 1).sum() == 1:
        p1 = rf.predict(X_test[np.where(y[indicies] == 1)[0][0]:np.where(y[indicies] == 1)[0][-1] + 1]).sum()/(y[indicies] == 1).sum()
    elif (y[indicies] == 1).sum() == 0:
        p1 = 10
    else:
        p1 = rf.predict(X_test[np.where(y[indicies] == 1)[0][0]:np.where(y[indicies] == 1)[0][-1]]).sum()/(y[indicies] == 1).sum()
    
    if (y[indicies] == 2).sum() == 1:
        p2 = rf.predict(X_test[np.where(y[indicies] == 2)[0][0]:np.where(y[indicies] == 2)[0][-1] + 1]).sum()/(y[indicies] == 2).sum()
    elif (y[indicies] == 2).sum() == 0:
        p2 = 10
    else:
        p2 = rf.predict(X_test[np.where(y[indicies] == 2)[0][0]:np.where(y[indicies] == 2)[0][-1]]).sum()/(y[indicies] == 2).sum()
        
    if (y[indicies] == 3).sum() == 1:
        p3 = rf.predict(X_test[np.where(y[indicies] == 3)[0][0]:np.where(y[indicies] == 3)[0][-1] + 1]).sum()/(y[indicies] == 3).sum()
    elif (y[indicies] == 3).sum() == 0:
        p3 = 10
    else:
        p3 = rf.predict(X_test[np.where(y[indicies] == 3)[0][0]:np.where(y[indicies] == 3)[0][-1]]).sum()/(y[indicies] == 3).sum()
        
    if (y[indicies] == 4).sum() == 1:
        p4 = rf.predict(X_test[np.where(y[indicies] == 4)[0][0]:np.where(y[indicies] == 4)[0][-1] + 1]).sum()/(y[indicies] == 4).sum()
    elif (y[indicies] == 4).sum() == 0:
        p4 = 10
    else:
        p4 = rf.predict(X_test[np.where(y[indicies] == 4)[0][0]:np.where(y[indicies] == 4)[0][-1]]).sum()/(y[indicies] == 4).sum()
                               
    if (y[indicies] == 5).sum() == 1:
        p5 = rf.predict(X_test[np.where(y[indicies] == 5)[0][0]:np.where(y[indicies] == 5)[0][-1] + 1]).sum()/(y[indicies] == 5).sum()
    elif (y[indicies] == 5).sum() == 0:
        p5 = 10
    else:
        p5 = rf.predict(X_test[np.where(y[indicies] == 5)[0][0]:np.where(y[indicies] == 5)[0][-1]]).sum()/(y[indicies] == 5).sum()
                               
    if (y[indicies] == 6).sum() == 1:
        p6 = rf.predict(X_test[np.where(y[indicies] == 6)[0][0]:np.where(y[indicies] == 6)[0][-1] + 1]).sum()/(y[indicies] == 6).sum()
    elif (y[indicies] == 6).sum() == 0:
        p6 = 10
    else:                        
        p6 = rf.predict(X_test[np.where(y[indicies] == 6)[0][0]:np.where(y[indicies] == 6)[0][-1]]).sum()/(y[indicies] == 6).sum()
                               
    if (y[indicies] == 7).sum() == 1:
        p7 = rf.predict(X_test[np.where(y[indicies] == 7)[0][0]:np.where(y[indicies] == 7)[0][-1] + 1]).sum()/(y[indicies] == 7).sum()
    elif (y[indicies] == 7).sum() == 0:
        p7 = 10
    else:
        p7 = rf.predict(X_test[np.where(y[indicies] == 7)[0][0]:np.where(y[indicies] == 7)[0][-1]]).sum()/(y[indicies] == 7).sum()
                               
    if (y[indicies] == 8).sum() == 1:
        p8 = rf.predict(X_test[np.where(y[indicies] == 8)[0][0]:np.where(y[indicies] == 8)[0][-1] + 1]).sum()/(y[indicies] == 8).sum()
    elif (y[indicies] == 8).sum() == 0:
        p8 = 10
    else:
        p8 = rf.predict(X_test[np.where(y[indicies] == 8)[0][0]:np.where(y[indicies] == 8)[0][-1]]).sum()/(y[indicies] == 8).sum()
    actual_placements = [0, p1, p2, p3, p4, p5, p6, p7, p8]
    predicted_placements = [0, p1, p2, p3, p4, p5, p6, p7, p8]
    predicted_placements.sort()
    avg += np.sum((predicted_placements - np.average(predicted_placements)) * (actual_placements - np.average(actual_placements)))/(8 * np.std(predicted_placements) * np.std(actual_placements))
    num_incorrect += (abs(predicted_placements.index(p1) - 1) + abs(predicted_placements.index(p2) - 2) + abs(predicted_placements.index(p3) - 3) + abs(predicted_placements.index(p4) - 4) + abs(predicted_placements.index(p5) - 5) + abs(predicted_placements.index(p6) - 6) + abs(predicted_placements.index(p7) - 7) + abs(predicted_placements.index(p8) - 8))
    games += 1
print("Accuracy = " + str(1 - num_incorrect/(36 * games)) + "      " + str(avg/games))

In [19]:
np.std(predicted_placements)

5.880608393667217

In [5]:
def twoSum(self, nums, target):
    mem = {}
        output = []
        for idx, num in enumerate(nums):
            if target-num in mem:
                return idx, mem[target-num]
            mem[num] = idx  

In [6]:
test()

0